In [1]:
from logging import Logger
import os
import logging
import sbol3
import labop
import tyto
import uml
import json
import rdflib as rdfl
from typing import Dict, Tuple
from sbol3 import Document

from labop.execution_engine import ExecutionEngine
from labop_convert.opentrons.opentrons_specialization import OT2Specialization, REVERSE_LABWARE_MAP
from labop_convert.markdown.markdown_specialization import MarkdownSpecialization

from IPython.display import Image

/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/sbol3/object.py:70: UserWarning: Using a default namespace. To set a namespace use set_namespace()
  warnings.warn('Using a default namespace.'


In [2]:
# Set up the document
print('Setting up document')
doc = sbol3.Document()
sbol3.set_namespace('https://bmey3labautomation.com/')

Setting up document


In [3]:
# Import the primitive libraries
print('Importing libraries')
labop.import_library('liquid_handling')
print('... Imported liquid handling')
labop.import_library('plate_handling')
print('... Imported plate handling')
labop.import_library('spectrophotometry')
print('... Imported spectrophotometry')
labop.import_library('sample_arrays')
print('... Imported sample arrays')
labop.import_library('pcr')
print('... Imported pcr')

Importing libraries
... Imported liquid handling
... Imported plate handling
... Imported spectrophotometry
... Imported sample arrays
... Imported pcr


In [4]:
protocol = labop.Protocol('labop_golden_gate_assembly')
protocol.name = "LabOP Golden Gate Assembly Protocol"
doc.add(protocol)

# Create the materials to be provisioned
CONT_NS = rdfl.Namespace('https://sift.net/container-ontology/container-ontology#')
OM_NS = rdfl.Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')

PREFIX_MAP = json.dumps({"cont": CONT_NS, "om": OM_NS})

In [5]:
# Define reagents
nfh2o = sbol3.Component('nfH2O', 'nucleasefreeh2o')
nfh2o.name = 'Nuclease-free H2O'
doc.add(nfh2o)

t4_dnaligase = sbol3.Component('t4_dnaligase', 't4dnaligasebuffer')
t4_dnaligase.name = 'T4 DNA Ligase Buffer'
doc.add(t4_dnaligase)

dest_plasmid = sbol3.Component('dest_plasmid', 'destinationplasmid')
dest_plasmid.name = 'Destination Plasmid'
doc.add(dest_plasmid)

dna_frag1 = sbol3.Component('dna_frag1', 'dnafragments1')
dna_frag1.name = 'DNA Fragment 1'
doc.add(dna_frag1)

dna_frag2 = sbol3.Component('dna_frag2', 'dnafragments2')
dna_frag2.name = 'DNA Fragment 2'
doc.add(dna_frag2)

gg_enzyme = sbol3.Component('gg_enzyme', 'ggenzymemix')
gg_enzyme.name = 'Golden Gate Enzyme Mix'
doc.add(gg_enzyme)

In [6]:
# Define labware
spec_reagent_plate = labop.ContainerSpec('reagent_plate',
                                    name='Well plate for reagents',
                                    queryString=REVERSE_LABWARE_MAP['masterblock_96_wellplate_2000ul'],
                                    prefixMap=PREFIX_MAP)
spec_destination_plate = labop.ContainerSpec('destination_plate',
                                        name='Destination plate',
                                        queryString=REVERSE_LABWARE_MAP['framestar_96_aluminumblock_200ul'],
                                        prefixMap=PREFIX_MAP)
spec_tiprack_p10 = labop.ContainerSpec('tiprack_p10',
                                       name='P10 Tiprack',
                                       queryString=REVERSE_LABWARE_MAP['opentrons_96_tiprack_10ul'],
                                        prefixMap=PREFIX_MAP)
spec_tiprack_p300 = labop.ContainerSpec('tiprack_p300',
                                        name='P300 Tiprack',
                                        queryString=REVERSE_LABWARE_MAP['opentrons_96_tiprack_300ul'],
                                        prefixMap=PREFIX_MAP)
p10_multi = sbol3.Agent('p10_multi',
                        name='P10 Multi')
p300_multi = sbol3.Agent('p300_multi',
                         name='P300 Multi')
thermocycler = sbol3.Agent('thermocycler_module',
                           name = 'Thermocycler Module')
doc.add(spec_tiprack_p10)
doc.add(spec_tiprack_p300)
doc.add(spec_reagent_plate)
doc.add(spec_destination_plate)
doc.add(p10_multi)
doc.add(p300_multi)
doc.add(thermocycler)

tiprack_p10 = protocol.primitive_step('EmptyContainer',
                                      specification=spec_tiprack_p10)
tiprack_p300 = protocol.primitive_step('EmptyContainer',
                                       specification=spec_tiprack_p300)
reagent_plate = protocol.primitive_step('EmptyContainer',
                                        specification=spec_reagent_plate)
destination_plate = protocol.primitive_step('EmptyContainer',
                                            specification=spec_destination_plate)

In [7]:
# Load OT2 robot with labware
left_pipette = protocol.primitive_step('ConfigureRobot', 
                                      instrument=p10_multi,
                                      mount='left')
right_pipette = protocol.primitive_step('ConfigureRobot',
                                        instrument=p300_multi,
                                        mount='right')
thermocycler_module = protocol.primitive_step('ConfigureRobot',
                                              instrument=thermocycler,
                                              mount='7')
load_tiprack_p10 = protocol.primitive_step('LoadRackOnInstrument',
                                           rack=spec_tiprack_p10,
                                           coordinates='1')
load_tiprack_p300 = protocol.primitive_step('LoadRackOnInstrument',
                                            rack=spec_tiprack_p300,
                                            coordinates='2')
load_reagent_plate = protocol.primitive_step('LoadRackOnInstrument',
                                             rack=spec_reagent_plate,
                                             coordinates='4')
load_destination_plate = protocol.primitive_step('LoadContainerOnInstrument',
                                                 specification=spec_destination_plate,
                                                 instrument=thermocycler,
                                                 slots='A1:H12') # 96 wells

In [8]:
# Set up target samples
assembly_reaction = protocol.primitive_step('PlateCoordinates',
                                            source=destination_plate.output_pin('samples'),
                                            coordinates='A1') # A1:H1 for multi-channel
negative_control = protocol.primitive_step('PlateCoordinates',
                                           source=destination_plate.output_pin('samples'),
                                           coordinates='A2') # A2:H2 for multi-channel

In [9]:
# Set up reagents - assuming reagents are loaded onto reagent plate manually
nfh2o_well = protocol.primitive_step('PlateCoordinates',
                                     source=reagent_plate.output_pin('samples'),
                                     coordinates='A1') # A1:H1 for multi-channel
t4_dnaligase_well = protocol.primitive_step('PlateCoordinates',
                                            source=reagent_plate.output_pin('samples'),
                                            coordinates='A2') # A2:H2 for multi-channel
dest_plasmid_well = protocol.primitive_step('PlateCoordinates',
                                            source=reagent_plate.output_pin('samples'),
                                            coordinates='A3') # A3:H3 for multi-channel
dna_frag1_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A4') # A3:H3 for multi-channel
dna_frag2_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A5') # A4:H4 for multi-channel
gg_enzyme_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A6') # A6:H6 for multi-channel

In [10]:
# Protocol
transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='right',
                                   amount=sbol3.Measure(28,tyto.OM.microliter))
transfer.name = '1. Transfer 28 ul of nuclease-free H2O (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='right',
                                   amount=sbol3.Measure(30,tyto.OM.microliter))
transfer.name = '2. Transfer 30 ul of nuclease-free H2O (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=t4_dnaligase_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(4,tyto.OM.microliter))
transfer.name = '3. Transfer 4 ul of T4 DNA ligase buffer (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=t4_dnaligase_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(4,tyto.OM.microliter))
transfer.name = '4. Transfer 4 ul of T4 DNA ligase buffer (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dest_plasmid_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '5. Transfer 2 ul of destination plasmid (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dest_plasmid_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '6. Transfer 2 ul of destination plasmid (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '7. Transfer 2 ul of DNA fragment 1 (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '8. Transfer 2 ul of DNA fragment 1 (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '9. Transfer 2 ul of DNA fragment 2 (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '10. Transfer 2 ul of DNA fragment 2 (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=gg_enzyme_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   pipette='left',
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '11. Transfer 2 ul of Golden Gate enzyme mix (assembly reaction).'

mix = protocol.primitive_step('PipetteMix',
                              samples=assembly_reaction.output_pin('samples'),
                              amount=sbol3.Measure(20,tyto.OM.microliter),
                              cycleCount='5',
                              pipette='right')
mix.name = '12. Mix assembly reaction (volume 20 ul, 5 repetitions)'

mix = protocol.primitive_step('PipetteMix',
                              samples=negative_control.output_pin('samples'),
                              amount=sbol3.Measure(20,tyto.OM.microliter),
                              cycleCount='5',
                              pipette='right')
mix.name = '13. Mix negative control (volume 20 ul, 5 repetitions)'

# incubate
incubate = protocol.primitive_step('Incubate',
                                   duration=sbol3.Measure(60,tyto.OM.minute),
                                   temperature=sbol3.Measure(37,tyto.OM.degree_Celsius))
incubate.name = '14. Incubate at 37 degrees for 60 minutes'

incubate = protocol.primitive_step('Incubate',
                                   duration=sbol3.Measure(5,tyto.OM.minute),
                                   temperature=sbol3.Measure(60,tyto.OM.degree_Celsius))
incubate.name = '15. Incubate at 60 degrees for 5 minutes'

In [11]:
filename = 'ot2_gg_labop_final'
agent = sbol3.Agent('ot2_machine', name='OT2 machine')
ee = ExecutionEngine(specializations=[OT2Specialization(filename)],failsafe=False)
parameter_values = []
execution = ee.execute(protocol, agent, id='test_execution')

https://bmey3labautomation.com/test_execution/CallBehaviorExecution20 https://bmey3labautomation.com/golden_gate_assembly_labop_v3_0/CallBehaviorAction20 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/CallBehaviorExecution20 https://bmey3labautomation.com/golden_gate_assembly_labop_v3_0/CallBehaviorAction20 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityEdgeFlow42 src = https://bmey3labautomation.com/golden_gate_assembly_labop_v3_0/CallBehaviorAction20/InputPin2 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityEdgeFlow67 src = https://bmey3labautomation.com/golden_gate_assembly_labop_v3_0/CallBehaviorAction20/InputPin1 param = source: http://bioprotocols.org/labop#SampleCollection 
https://bmey3labautomation.com/test_execution/ActivityNodeExecution28 https://bmey3labautomation.com/golde